# TODO
- функция для быстрого расчета геометрии по полуфрейму
- Написать и оптимизировать функцию определения координат особых точек
- написать функцию отклонения от расстояний

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'..')
import pynamod
from pynamod.parsers_3dna import parse_bp_par_file
from pynamod.energy_constants import get_consts_olson_98,get_dye_data
from pynamod.utils import get_movable_steps

In [2]:
AVERAGE,FORCE_CONST,DISP=get_consts_olson_98()
header,pairtypes,full_par_frame=parse_bp_par_file('Resources/1kx5.par')

Processing  Resources/1kx5.par


In [3]:
#testing the speed of main energy func
from pynamod.energy_funcs import get_force_matrix, get_average_bpstep_frame, get_bpstep_frame, _calc_bend_energy

movable_steps=get_movable_steps([[130,147]])
force_matrix=get_force_matrix(pairtypes,movable_steps,FORCE_CONST)
average_bpstep_frame=get_average_bpstep_frame(pairtypes,movable_steps,AVERAGE)
bpstep_frame=get_bpstep_frame(full_par_frame,movable_steps)

print(_calc_bend_energy(bpstep_frame,force_matrix,average_bpstep_frame))
%timeit _calc_bend_energy(bpstep_frame,force_matrix,average_bpstep_frame)

83.76470270950003
10.7 µs ± 31.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
#testing the speed of decorated energy func
from pynamod.utils import get_movable_steps
from pynamod.energy_funcs import get_bend_energy
movable_steps=get_movable_steps([[130,147]])

print(get_bend_energy(full_par_frame,pairtypes,movable_steps,FORCE_CONST,AVERAGE))
%timeit get_bend_energy(full_par_frame,pairtypes,movable_steps,FORCE_CONST,AVERAGE)

83.76470270950003
237 µs ± 278 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
# minimization routines
from pynamod.optimizers import run_basinhopping
movable_steps=get_movable_steps([[50,100]])
new_full_par_frame,res=run_basinhopping(full_par_frame,pairtypes,movable_steps,FORCE_CONST,AVERAGE,
                                       options={'niter':5,'T': 1000},method='Powell')

basinhopping step 0: f 9.81717e-21
basinhopping step 1: f 1.00635e-20 trial_f 1.00635e-20 accepted 1  lowest_f 9.81717e-21
basinhopping step 2: f 9.86192e-21 trial_f 9.86192e-21 accepted 1  lowest_f 9.81717e-21
basinhopping step 3: f 7.97375e-21 trial_f 7.97375e-21 accepted 1  lowest_f 7.97375e-21
found new global minimum on step 3 with function value 7.97375e-21
basinhopping step 4: f 8.16089e-21 trial_f 8.16089e-21 accepted 1  lowest_f 7.97375e-21
basinhopping step 5: f 8.3465e-21 trial_f 8.3465e-21 accepted 1  lowest_f 7.97375e-21


In [6]:
from pynamod.bp_step_geometry import rebuild_by_full_par_frame_numba
from pynamod.visual_ngl import show_ref_frames
from pynamod.non_DNA_geometry import get_obj_orientation,get_obj_orientation_and_location
from pynamod.geometry_transformations import length

_ColormakerRegistry()

In [17]:
bp_frames=rebuild_by_full_par_frame_numba(new_full_par_frame)

In [19]:
view=show_ref_frames(bp_frames)

In [20]:
view

NGLWidget()

In [12]:
dye_data=get_dye_data()

In [13]:
print('Available dyes are: ',dye_data['name'])

Available dyes are:  ['cy3_forward' 'cy3_reverse' 'cy5_forward' 'cy5_reverse']


In [14]:
import numpy as np
def get_distances(full_par_frame,pairs=[[0,10],[0,11]],types=[[dye_data[0],dye_data[3]],[dye_data[0],dye_data[3]]]):
    bp_ref_frames=rebuild_by_full_par_frame_numba(full_par_frame)
    results=np.zeros(len(pairs))
    for i,(pair,dye_types) in enumerate(zip(pairs,types)):
        calcO1=get_obj_orientation(bp_ref_frames[pair[0],:3,:3],bp_ref_frames[pair[0],3,:3],dye_types[0]['Of_vec'])
        calcO2=get_obj_orientation(bp_ref_frames[pair[1],:3,:3],bp_ref_frames[pair[1],3,:3],dye_types[1]['Of_vec'])
        results[i]=length(calcO1-calcO2)
    return results
#%timeit get_distances(new_full_par_frame)
get_distances(new_full_par_frame)
def _calc_dist_deviation(full_par_frame,pairs=[[0,10],[0,11]],types=[[dye_data[0],dye_data[3]],[dye_data[0],dye_data[3]]],dists=[30,34],weights=[1,1]):
    distances=get_distances(full_par_frame,pairs,types)
    return np.sum(weights*(dists-distances)**2)
_calc_dist_deviation(new_full_par_frame[:])

43.75184828355875

In [15]:
items = [1, 2, 3, 4, 5]
squared = list(map(lambda x: x**2, items))

In [21]:
calcR,calcO=get_obj_orientation_and_location(bp_frames[100,:3,:3],bp_frames[100,3,:3],dye_data[0]['R_mat'],dye_data[0]['Of_vec'])
view.shape.add_sphere(calcO.tolist(),[0,0,1],4)
view

NGLWidget(n_components=3)